In [4]:
import json
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_and_parse(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.select_one('body > div > main > div:nth-of-type(2) > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(2) > div > div:nth-of-type(1) > a > h1')
        if not title:
            return None, None

        target_div = soup.select_one('body > div > main > div:nth-of-type(2) > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(3)')
        paragraphs = [p.get_text() for p in target_div.find_all('p')] if target_div else []

        return title.get_text(), paragraphs
    else:
        print(f"failed to retrieve the page: {url}. Status code: {response.status_code}")
        return None, None

def main():
    with open("links.json", 'r') as file:
        data = json.load(file)

    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(fetch_and_parse, url): url for url in data['links']}

        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                title, paragraphs = future.result()
                if title and paragraphs is not None:
                    print(f'[ Title ]: {title}')
                    print(f'[ Paragraphs ]: {paragraphs}\n\n')
                else:
                    print(f"no content found for {url}.")
            except Exception as e:
                print(f"{url}: {e}")

if __name__ == '__main__':
    main()


[ Title ]: Insiden maut pria pendiam yang ingin memisahkan burger beku
[ Paragraphs ]: ['Seorang pria Wales menghilang selama sepekan sebelum akhirnya ditemukan. Ternyata ia tidak kemana-mana hanya di kamar tidurnya. Namun, saat ditemukan sudah tak bernyawa.\xa0 Kematiannya itu terkait dengan keinginannya menyantap burger.\xa0\xa0', 'Barry Griffiths ditemukan di flatnya di Llandrindod Wells pada tanggal 4 Juli 2023. Pengadilan Pemeriksa Mayat Pontypridd mendengar bahwa polisi menemukan Griffiths, seorang "pria yang sangat tertutup", terbaring berpakaian lengkap di tempat tidurnya dengan darah di perutnya yang bengkak.', 'Darah ditemukan di seluruh flat tetapi ponsel, dompet, dan komputer Griffiths tidak tersentuh dan tidak ada tanda-tanda gangguan apa pun.', 'Hasil otopsi kemudian menetapkan bahwa Griffiths menderita satu luka tusuk.', 'Sersan Detektif Stephen Vaughan mengatakan kepada pemeriksaan bahwa ia datang ke tempat kejadian perkara dan meyakini bahwa itu adalah "kematian yang t

In [5]:
import json
import requests
from bs4 import BeautifulSoup
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_and_parse(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            title_element = soup.select_one('body > div > main > div:nth-of-type(2) > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(2) > div > div:nth-of-type(1) > a > h1')

            if title_element:
                title = title_element.get_text()
                target_div = soup.select_one('body > div > main > div:nth-of-type(2) > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(3)')

                if target_div:
                    paragraphs = [p.get_text() for p in target_div.find_all('p')]
                    content = ' '.join(paragraphs)
                    return {'title': title, 'url': url, 'content': content}
                else:
                    print(f"target div not found for URL: {url}")
            else:
                print(f"title element not found for URL: {url}")
        else:
            print(f"failed to retrieve the page. status code: {response.status_code} for URL: {url}")
    except Exception as e:
        print(f"{url} :{e}")

    return None

def main():
    with open("links.json", 'r') as file:
        data = json.load(file)

    with open("datasets.csv", 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['title', 'url', 'content']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        with ThreadPoolExecutor(max_workers=10) as executor:
            future_to_url = {executor.submit(fetch_and_parse, url): url for url in data['links']}
            for future in as_completed(future_to_url):
                result = future.result()
                if result:
                    writer.writerow(result)
                    print(result)

if __name__ == '__main__':
    main()


{'title': 'Air mata dan perjuangan keluarga Kristen Palestina di Betlehem melawan pemukim Israel', 'url': 'https://www.alinea.id/peristiwa/keluarga-kristen-di-betlehem-melawan-pemukim-israel-b2kEu9Q31', 'content': 'Selama 12 tahun, keluarga Kisiya dari Palestina telah berjuang untuk mempertahankan sebidang tanah milik mereka di desa al-Makhrour yang secara historis merupakan desa Kristen, yang terletak di kota pegunungan Beit Jala yang menakjubkan di dekat Betlehem. “Kami memiliki dokumen Palestina dan Israel untuk membuktikan kepemilikan kami atas tanah ini,” kata Alice Kisiya, putri pemilik tanah dan aktivis terkemuka yang menentang pemukiman ilegal Israel, kepada The New Arab. Minggu lalu, pasukan Israel menghancurkan perkemahan protes yang didirikan oleh Kisiya dengan dukungan dari solidaritas internasional dan aktivis perdamaian Israel di dekat lahan seluas 1.000 meter persegi yang disita secara ilegal oleh pemukim Israel pada tanggal 31 Juli yang didukung oleh militer Israel, men